In [387]:
import pandas as pd
import numpy as np
import json
import os

In [388]:
with open('contacts.json') as file:
    data = json.load(file)

### dict of objects
- 3 dicts : Email , Phone , OrderId
1. iterate all contacts in any dict
2. use other dicts to find matches

In [389]:
class Customer():
    def __init__(self, **kwargs):
        self.customerId = kwargs['Id']
        self.email = kwargs['Email']
        self.phone = kwargs['Phone']
        self.orderId = kwargs['OrderId']
        self.contacts = kwargs['Contacts']
        
    def __str__(self):
        return str(self.customerId)

In [390]:
customer_db = []
for values in data:
    customer_db.append(Customer(**values))

In [391]:
email_dict = {}
for customer in customer_db:
    if customer.email != '':
        if customer.email not in email_dict:
            email_dict[customer.email] = [customer]
        else:
            email_dict[customer.email].append(customer)

In [392]:
phone_dict = {}
for customer in customer_db:
    if customer.phone != '':
        if customer.phone not in phone_dict:
            phone_dict[customer.phone] = [customer]
        else:
            phone_dict[customer.phone].append(customer)

In [393]:
order_dict = {}
for customer in customer_db:
    if customer.orderId != '':
        if customer.orderId not in order_dict:
            order_dict[customer.orderId] = [customer]
        else:
            order_dict[customer.orderId].append(customer)

In [394]:
# Iterate through EMAIL
same_customer_email =[]
for email in email_dict.keys():
    temp = set(email_dict[email])
    for customer in email_dict[email]:
        # Match from PHONE
        if customer.phone:
            temp.update(set(phone_dict[customer.phone]))
        # Match from ORDER
        if customer.orderId:
            temp.update(set(order_dict[customer.orderId]))
    same_customer_email.append(temp)

In [395]:
# Iterate through PHONE
same_customer_phone =[]
for phone in phone_dict.keys():
    temp = set(phone_dict[phone])
    for customer in phone_dict[phone]:
        # Match from EMAIL
        if customer.email:
            temp.update(set(email_dict[customer.email]))
        # Match from ORDER
        if customer.orderId:
            temp.update(set(order_dict[customer.orderId]))
    same_customer_phone.append(temp)

In [396]:
# Iterate through ORDER
same_customer_order =[]
for order in order_dict.keys():
    temp = set(order_dict[order])
    for customer in order_dict[order]:
        # Match from PHONE
        if customer.phone:
            temp.update(set(phone_dict[customer.phone]))
        # Match from EMAIL
        if customer.email:
            temp.update(set(email_dict[customer.email]))
    same_customer_order.append(temp)

In [397]:
sum([len(i) for i in same_customer_email]), sum([len(i) for i in same_customer_phone]), sum([len(i) for i in same_customer_order])

(493808, 380354, 426315)

In [398]:
class NewCustomer():
    def __init__(self, customerList):
        self.id = sorted(customerList, key=lambda x: x.customerId)[0].customerId
        self.trace = '-'.join([str(i) for i in sorted(customerList, key=lambda x: x.customerId)])
        self.contact = sum([i.contacts for i in customerList])
        
    def __str__(self):
        return '_'.join([str(self.id), self.trace, str(self.contact)])

In [400]:
soln_email = [NewCustomer(i) for i in same_customer_email]
soln_phone = [NewCustomer(i) for i in same_customer_phone]
soln_order = [NewCustomer(i) for i in same_customer_order]

In [401]:
len(soln_email), len(soln_phone), len(soln_order), len(soln_email)+len(soln_phone)+len(soln_order)

(249156, 190677, 189302, 629135)

In [402]:
# byEmail = [(i.id, i.trace, i.contact) for i in soln_email]
# byPhone = [(i.id, i.trace, i.contact) for i in soln_phone]
# byOrder = [(i.id, i.trace, i.contact) for i in soln_order]

In [403]:
allCustomers = sorted(byEmail + byPhone + byOrder, key=lambda x: (x[0], len(x[1])))
allCustomers

[(0, '0', 1),
 (1, '1-2458-476346', 5),
 (1, '1-2458-476346', 5),
 (1, '1-2458-98519-115061-476346', 11),
 (1, '1-2458-98519-115061-476346', 11),
 (1, '1-2458-98519-115061-165605-476346', 11),
 (2, '2-159312-322639-348955', 4),
 (2, '2-159312-322639-348955', 4),
 (3, '3', 0),
 (3, '3', 0),
 (4, '4', 2),
 (5, '5-50-226720-458692', 8),
 (5, '5-50-226720-458692', 8),
 (5, '5-50-215197-226720-404324-458692-482810', 15),
 (6, '6-38-32871-142067', 10),
 (6, '6-38-32871-142067', 10),
 (6, '6-38-32871-142067-236367', 13),
 (6, '6-38-32871-142067-236367', 13),
 (6, '6-38-32871-142067-236367', 13),
 (7, '7', 1),
 (7, '7', 1),
 (8, '8-183160', 5),
 (8, '8-183160-406623', 5),
 (8, '8-183160-406623', 5),
 (8, '8-183160-406623', 5),
 (9, '9-13-343161', 1),
 (9, '9-13-16708-343161-417916', 5),
 (9, '9-13-16708-343161-417916', 5),
 (9, '9-13-33415-343161-468927-484896', 4),
 (9, '9-13-16708-343161-417916-468927-484896', 7),
 (10, '10-93270', 7),
 (10, '10-93270', 7),
 (10, '10-93270', 7),
 (11, '11-24

In [404]:
soln = {}
for item in allCustomers:
    if item not in soln.keys():
        soln[item[0]] = (item[1], item[2])
    else:
        if len(soln[item[0]][0]) < len(item[1]):
            soln[item[0]] = (item[1], item[2])

In [405]:
soln_dict = {
    'ticket_id': [],
    'ticket_trace/contact': [],
}
for item in soln.items():
    soln_dict['ticket_id'].append(item[0])
    soln_dict['ticket_trace/contact'].append(item[1][0] + f', {item[1][1]}')

In [406]:
soln_df = pd.DataFrame(soln_dict)
soln_df.to_csv('ans.csv', index=False)

In [407]:
pd.read_csv('ans.csv')

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-165605-476346, 11"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
329416,499993,"499993, 4"
329417,499995,"499995, 2"
329418,499996,"499996, 4"
329419,499997,"499997, 2"
